<a href="https://colab.research.google.com/github/ipunn/Kaggle_Titanic/blob/main/TITANIC_Auto_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
train_set_raw = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Kaggle-Titanic/train.csv")
test_set_raw = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Kaggle-Titanic/test.csv")

In [ ]:
!pip install flaml
!pip install flaml[notebook]

     |████████████████████████████████| 215 kB 5.1 MB/s 
     |████████████████████████████████| 22.3 MB 952 kB/s 
     |████████████████████████████████| 67.4 MB 73 kB/s 
     |████████████████████████████████| 2.0 MB 59.1 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3
     |████████████████████████████████| 158 kB 5.4 MB/s 
     |████████████████████████████████| 757 kB 43.0 MB/s 
     |████████████████████████████████| 2.6 MB 32.8 MB/s 
     |████████████████████████████████| 12.4 MB 164 kB/s 
  Created wheel for openml: filename=openml-0.10.2-py3-none-any.whl size=190318 sha256=e8ca1e51ecee970b62724ca8269cf3a12e7d9aaf8678fb898835ab13a82877c6
  Stored in directory: /root/.

In [ ]:
train_set = pd.DataFrame()
test_set = pd.DataFrame()

train_set = train_set_raw[['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
train_set = train_set.dropna(axis=0,how='any')

X = train_set.drop(columns=['Survived'])
X = pd.get_dummies(X)
y = train_set.Survived

predict_set = test_set_raw[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
predict_set = pd.get_dummies(predict_set)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3 , random_state=44)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((498, 10), (214, 10), (498,), (214,))

In [ ]:
y_train

113    0
883    0
204    1
91     0
812    0
      ..
110    0
123    1
720    1
217    0
529    0
Name: Survived, Length: 498, dtype: int64

In [ ]:
from flaml import AutoML
automl = AutoML()

# Specify automl goal and constraint
automl_settings = {
    "metric": 'accuracy',
    "task": 'classification',
}
automl.fit(X_train, y_train,**automl_settings)

[flaml.automl: 08-22 14:18:03] {1121} INFO - Evaluation method: cv
[flaml.automl: 08-22 14:18:03] {618} INFO - Using StratifiedKFold
[flaml.automl: 08-22 14:18:03] {1142} INFO - Minimizing error metric: 1-accuracy
[flaml.automl: 08-22 14:18:03] {1163} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'catboost', 'xgboost', 'extra_tree', 'lrl1']
[flaml.automl: 08-22 14:18:03] {1253} INFO - iteration 0, current learner lgbm
[flaml.automl: 08-22 14:18:03] {1411} INFO -  at 0.1s,	best lgbm's error=0.2008,	best lgbm's error=0.2008
[flaml.automl: 08-22 14:18:03] {1253} INFO - iteration 1, current learner lgbm
[flaml.automl: 08-22 14:18:03] {1411} INFO -  at 0.2s,	best lgbm's error=0.2008,	best lgbm's error=0.2008
[flaml.automl: 08-22 14:18:03] {1253} INFO - iteration 2, current learner lgbm
[flaml.automl: 08-22 14:18:03] {1411} INFO -  at 0.3s,	best lgbm's error=0.1987,	best lgbm's error=0.1987
[flaml.automl: 08-22 14:18:03] {1253} INFO - iteration 3, current learner lgbm
[flaml.autom

In [ ]:
# # Predict
y_pred = automl.predict(X_test)

from sklearn.metrics import accuracy_score
print("Accuracy :", accuracy_score(y_test,y_pred))

# Export the best model
print(automl.model)
print(automl.model.estimator)

Accuracy : 0.8177570093457944


In [ ]:
''' retrieve best config and best learner'''
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best accuracy on validation data: {0:.4g}'.format(1-automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: catboost
Best hyperparmeter config: {'early_stopping_rounds': 10, 'learning_rate': 0.06233639237958607}
Best accuracy on validation data: 0.8233
Training duration of best run: 0.6214 s


In [ ]:
prediction = pd.DataFrame(automl.predict(predict_set))

output = test_set_raw[['PassengerId']].join(prediction)

output.columns = ['PassengerId', 'Survived']

output.to_csv('/content/drive/MyDrive/Colab Notebooks/Kaggle-Titanic/prediction_flaml2.csv',index=False)